# 🧪 Lab Supply Calculator

Welcome! Use this tool to determine the quantity of supplies you will need for a given sample size, compare that to the current inventory list, and generate a report of supply needs and reorder status.

---

## 🧭 Instructions

1.  Press "**Run all**" to get started.
2.  Enter your **name** (first initial and last name, e.g., AOnoufriou).
3.  Enter the **sample size**.
4.  Choose the **protocol** from the dropdown menu.
5.  Click "**Generate Report**".
6.  Click "**Download CSV**" to save the report.

In [1]:
#@title Install packages and imports
##Install packages and imports
# Installing packages
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import yaml
except ImportError:
    install_package("pyyaml")
import yaml

try:
    import gspread
except ImportError:
    install_package("gspread")

# Ensure these are installed for Google Colab authentication with gspread
try:
    import google.auth
except ImportError:
    install_package("google-auth")
try:
    import google.auth.transport.requests
except ImportError:
    install_package("google-auth-httplib2") # or google-auth-oauthlib, httplib2 is more common for initial setup


# Import for Google Colab authentication
try:
    from google.colab import auth
except ImportError:
    print("Warning: google.colab.auth not found. Running outside Colab might require manual authentication setup for gspread.")


# Import all required libraries
import requests
import csv
import pandas as pd
import os
from io import StringIO
from google.colab import files
from IPython.display import display, clear_output
import ipywidgets as widgets
import time

print("✅ All packages installed and imported successfully!")

# --- Google Sheet Authentication Sanity Check (Moved Here) ---
global gc # Make gc global so it can be used in load_inventory
try:
    print("\n🔑 Authenticating Google Colab for Sheets access (this may open a pop-up)...")
    auth.authenticate_user()
    creds, _ = google.auth.default()
    gc = gspread.authorize(creds)
    print("✅ Google Sheets authentication successful!")
except Exception as e:
    print(f"❌ Initial Google Sheets authentication failed: {e}")
    print("Please ensure you complete the authentication pop-up and have proper sheet permissions.")
    gc = None # Set gc to None if authentication fails
# ----------------------------------------------------------

# Define GitHub repo configuration
GITHUB_USER = "aono87"
REPO_NAME = "inventory_test"
BRANCH = "main"
PROTOCOL_DIR = "test_protocol_check/protocols"
GOOGLE_SHEET_URL = "https://docs.google.com/spreadsheets/d/1uJeollRVlBDNcQU-FnPCYjLaCHbbOND2OEqcm3eKd2M/edit?usp=sharing"

print("✅ Configuration set")

#Helper Functions
def github_raw_url(path):
    return f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/{BRANCH}/{path}"

def fetch_file(path):
    url = github_raw_url(path)
    # No token is needed for a public repository
    try:
        r = requests.get(url)
        r.raise_for_status()
        return r.text
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching {path}: {e}")
        raise

def get_protocol_list(user, repo, path, branch="main"):
    url = f"https://api.github.com/repos/{user}/{repo}/contents/{path}?ref={branch}"
    # No token is needed for a public repository
    try:
        r = requests.get(url)
        r.raise_for_status()
        files = r.json()
        return [f['path'] for f in files if f['name'].endswith('.yaml')]
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching protocol list: {e}")
        return []

def load_inventory():
    """
    Loads inventory data from the specified Google Sheet.
    Assumes the first worksheet contains the inventory data.
    """
    if gc is None:
        print("❌ Google Sheets client not initialized. Cannot load inventory.")
        return {}

    try:
        spreadsheet = gc.open_by_url(GOOGLE_SHEET_URL)
        worksheet = spreadsheet.get_worksheet(0)  # Get the first worksheet

        # Get all records as a list of dictionaries
        records = worksheet.get_all_records()

        inv = {}
        for row in records:
            item = row.get('Item', '').strip()
            if not item: # Skip rows that don't have an item name
                continue
            try:
                inv[item] = {
                    'unit': row.get('Unit', '').strip(),
                    'stock': float(row.get('Stock Quantity', 0)),
                    'threshold': float(row.get('Reorder Threshold', 0))
                }
            except ValueError as ve:
                print(f"⚠️ Skipping row due to data conversion error for item '{item}': {ve}")
                continue
        print("✅ Inventory loaded successfully from Google Sheet.")
        return inv
    except Exception as e:
        print(f"❌ Error loading inventory from Google Sheet: {e}")
        print("Please ensure the Google Sheet is shared correctly (e.g., 'Anyone with the link can view').")
        return {}

def load_protocol(text):
    try:
        return yaml.safe_load(text)
    except Exception as e:
        print(f"❌ Error loading protocol: {e}")
        return {}

def calculate_needs(protocol_data, sample_count):
    if 'supplies_per_sample' not in protocol_data:
        print("❌ Protocol missing 'supplies_per_sample' section")
        return {}
    return {item: qty * sample_count for item, qty in protocol_data['supplies_per_sample'].items()}

print("✅ Helper functions defined")

✅ All packages installed and imported successfully!

🔑 Authenticating Google Colab for Sheets access (this may open a pop-up)...
✅ Google Sheets authentication successful!
✅ Configuration set
✅ Helper functions defined


In [2]:
#@title Generate Supply Report & Update Sheet
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
import os
import csv

print("🚀 Starting Lab Supply Calculator...")
print("=" * 50)

# --- Global variables to hold report data and control actions ---
report_data_for_csv = {}
is_downloading = False

# --- Step 1: Fetch protocols and create input widgets ---
print("🔍 Fetching available protocols from GitHub...")
try:
    protocols = get_protocol_list(GITHUB_USER, REPO_NAME, PROTOCOL_DIR, BRANCH)
    if not protocols:
        print("⚠️ No protocols found on GitHub, using default list.")
        protocols = [
            "test_protocol_check/protocols/dna_extraction_mn.yaml",
            "test_protocol_check/protocols/pcr_setup.yaml"
        ]
    else:
        print(f"✅ Found {len(protocols)} protocols.")
except Exception as e:
    print(f"❌ Failed to fetch protocols: {e}. Using default list.")
    protocols = [
        "test_protocol_check/protocols/dna_extraction_mn.yaml",
        "test_protocol_check/protocols/pcr_setup.yaml"
    ]

# Create input widgets
name_input = widgets.Text(
    value='',
    placeholder='e.g., AOnoufriou',
    description='Your Name:',
    style={'description_width': 'initial'}
)

sample_input = widgets.IntText(
    value=50,
    description='Sample size:',
    style={'description_width': 'initial'}
)

protocol_dropdown = widgets.Dropdown(
    options=protocols,
    description='Protocol:',
    layout=widgets.Layout(width='95%'),
    style={'description_width': 'initial'}
)

generate_button = widgets.Button(
    description='Generate Report',
    button_style='primary',
    tooltip='Click to generate the supply report',
    layout=widgets.Layout(width='200px', margin='0 10px 0 0')
)

update_sheet_button = widgets.Button(
    description='Update Inventory Sheet',
    button_style='warning', # A different color to distinguish
    tooltip='Writes the hold information to the Google Sheet',
    layout=widgets.Layout(width='200px', margin='0 10px 0 0', display='none') # Hidden initially
)

download_button = widgets.Button(
    description='Download CSV',
    button_style='success',
    tooltip='Click to download the report as a CSV file',
    layout=widgets.Layout(width='200px', display='none') # Hidden initially
)

# Output area for results
output_area = widgets.Output()

# --- Step 2: Define the functions for generating, updating, and downloading ---
def generate_report(b):
    """Generate the supply report and display it."""
    with output_area:
        output_area.clear_output()
        # Reset and hide action buttons at the start of generation
        download_button.layout.display = 'none'
        update_sheet_button.layout.display = 'none'
        generate_button.disabled = True
        generate_button.description = "Generating..."
        generate_button.button_style = 'info'

        # Input validation
        if not name_input.value:
            print("❌ Please enter your name.")
            generate_button.disabled = False
            generate_button.description = "Generate Report"
            generate_button.button_style = 'primary'
            return
        if sample_input.value <= 0:
            print("❌ Please enter a sample size greater than zero.")
            generate_button.disabled = False
            generate_button.description = "Generate Report"
            generate_button.button_style = 'primary'
            return

        try:
            print("🔄 Generating report...")
            inventory = load_inventory()
            protocol_text = fetch_file(protocol_dropdown.value)
            protocol = load_protocol(protocol_text)

            if not inventory or not protocol:
                raise Exception("Failed to load inventory or protocol data.")

            needs = calculate_needs(protocol, sample_input.value)
            if not needs:
                raise Exception("Failed to calculate supply needs.")

            print(f"✅ Calculated needs for {len(needs)} items")

            report_rows = []
            per_sample = protocol.get("supplies_per_sample", {})
            for item, total_required in needs.items():
                inv = inventory.get(item)
                if not inv:
                    report_rows.append({"Item": item, "Per Sample": per_sample.get(item, "N/A"), "Need": total_required, "Stock": "N/A", "Status": "MISSING", "Reorder": "⚠️"})
                else:
                    status = "OK" if inv['stock'] >= total_required else "LOW"
                    reorder_flag = "YES" if (inv['stock'] - total_required) < inv['threshold'] else "NO"
                    report_rows.append({"Item": item, "Per Sample": per_sample.get(item, "N/A"), "Need": round(total_required, 2), "Stock": round(inv['stock'], 2), "Status": status, "Reorder": reorder_flag})

            df = pd.DataFrame(report_rows)
            protocol_name = os.path.basename(protocol_dropdown.value)

            # Store data for CSV download and sheet update
            report_data_for_csv['df'] = df
            report_data_for_csv['user_name'] = name_input.value
            report_data_for_csv['protocol_name'] = protocol_name
            report_data_for_csv['sample_size'] = sample_input.value

            # Display report
            print(f"\n📋 Supply Report for {sample_input.value} samples by {name_input.value}")
            print("=" * 80)
            display(df)
            print("\n✅ Report generated successfully!")

            # Show and enable the action buttons
            download_button.description = 'Download CSV'
            download_button.disabled = False
            download_button.layout.display = ''
            update_sheet_button.description = 'Update Inventory Sheet'
            update_sheet_button.disabled = False
            update_sheet_button.layout.display = ''

        except Exception as e:
            print(f"❌ Error: {e}")

        finally:
            # Re-enable the generate button for another run
            generate_button.description = "Generate Another Report"
            generate_button.button_style = "primary"
            generate_button.disabled = False

def update_inventory_sheet(b):
    """Writes the hold information from the report to the Google Sheet."""
    with output_area:
        print("\n🔄 Updating Google Sheet with hold information...")
        update_sheet_button.disabled = True
        update_sheet_button.description = "Updating..."

        try:
            # Retrieve stored data
            report_df = report_data_for_csv.get('df')
            user_name = report_data_for_csv.get('user_name', 'N/A')
            protocol_name = report_data_for_csv.get('protocol_name', 'N/A')

            if report_df is None:
                print("❌ No report data found to update.")
                return

            # --- Google Sheet Update Logic ---
            spreadsheet = gc.open_by_url(GOOGLE_SHEET_URL)
            worksheet = spreadsheet.get_worksheet(0)

            # Get all current data to find rows and columns efficiently
            headers = worksheet.row_values(1)
            inventory_data = worksheet.get_all_records()

            # Create a map of Item Name -> its row index in the sheet
            # (add 2 to the index: 1 for header, 1 for 1-based indexing)
            item_to_row_map = {str(record['Item']).strip(): i + 2 for i, record in enumerate(inventory_data)}

            # Find column indices for the 'hold' columns
            hold_for_col = headers.index('Hold For') + 1
            hold_amount_col = headers.index('Hold Amount') + 1
            hold_date_col = headers.index('Hold Date') + 1
            hold_project_col = headers.index('Hold Project') + 1

            date_str = pd.Timestamp.now().strftime('%Y-%m-%d')
            updates = []

            # Prepare a list of updates to send in a batch
            for _, row in report_df.iterrows():
                item_name = str(row['Item']).strip()
                needed_amount = row['Need']

                if item_name in item_to_row_map:
                    row_idx = item_to_row_map[item_name]
                    updates.append({'range': f'{gspread.utils.rowcol_to_a1(row_idx, hold_for_col)}', 'values': [[user_name]]})
                    updates.append({'range': f'{gspread.utils.rowcol_to_a1(row_idx, hold_amount_col)}', 'values': [[str(needed_amount)]]})
                    updates.append({'range': f'{gspread.utils.rowcol_to_a1(row_idx, hold_date_col)}', 'values': [[date_str]]})
                    updates.append({'range': f'{gspread.utils.rowcol_to_a1(row_idx, hold_project_col)}', 'values': [[protocol_name]]})
                else:
                    print(f"⚠️ Item '{item_name}' not found in inventory sheet. Skipping update for this item.")

            if updates:
                worksheet.batch_update(updates)
                print(f"✅ Successfully updated hold information for {len(updates)//4} items in the Google Sheet.")
                update_sheet_button.description = 'Updated ✓'
            else:
                print("🤷 No items in the report matched the inventory sheet. Nothing to update.")
                update_sheet_button.description = 'Nothing to Update'

        except gspread.exceptions.APIError as e:
            print(f"❌ A Google API error occurred: {e}")
            print("   Please ensure you have 'Editor' permissions for the Google Sheet.")
            update_sheet_button.disabled = False
            update_sheet_button.description = 'Update Failed'
        except Exception as e:
            print(f"❌ An unexpected error occurred: {e}")
            update_sheet_button.disabled = False
            update_sheet_button.description = 'Update Failed'

def download_csv(b):
    """Download the generated report data as a single CSV file with metadata."""
    global is_downloading

    if is_downloading:
        return

    is_downloading = True
    download_button.disabled = True

    try:
        with output_area:
            print("\n💾 Preparing CSV for download...")

            # Retrieve stored data
            df = report_data_for_csv.get('df')
            user_name = report_data_for_csv.get('user_name', 'N/A')
            protocol_name = report_data_for_csv.get('protocol_name', 'N/A')
            sample_size = report_data_for_csv.get('sample_size', 'N/A')

            if df is None:
                print("❌ No report data found to download.")
                return

            # Sanitize and create filename
            protocol_base_name = os.path.splitext(protocol_name)[0].replace(' ', '_')
            date_str = pd.Timestamp.now().strftime('%Y-%m-%d')
            filename = f"Report_{protocol_base_name}_{sample_size}-samples_{user_name}_{date_str}.csv"
            output_path = f"/content/{filename}"

            # Add metadata rows as DataFrame
            metadata = pd.DataFrame({
                'Item': [f"User: {user_name}", f"Protocol: {protocol_name}", f"Sample Size: {sample_size}", f"Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}"],
                'Per Sample': [''] * 4,
                'Need': [''] * 4,
                'Stock': [''] * 4,
                'Status': [''] * 4,
                'Reorder': [''] * 4
            })

            # Concatenate metadata + actual report
            final_output = pd.concat([metadata, df], ignore_index=True)

            # Write final CSV
            final_output.to_csv(output_path, index=False)

            print(f"📁 Starting download for: {filename}")
            files.download(output_path)
            print("🎉 Download complete!")
            download_button.description = 'Downloaded ✓'

    except Exception as e:
        with output_area:
            print(f"❌ Could not download file. Error: {e}")
        download_button.disabled = False
    finally:
        is_downloading = False



# --- Step 3: Connect buttons to functions and display widgets ---
generate_button.on_click(generate_report)
update_sheet_button.on_click(update_inventory_sheet)
download_button.on_click(download_csv)

buttons_box = widgets.HBox([generate_button, update_sheet_button, download_button])
print("📝 Please enter your details below (first initial and last name, e.g., AOnoufriou).")
display(name_input, sample_input, protocol_dropdown, buttons_box, output_area)


🚀 Starting Lab Supply Calculator...
🔍 Fetching available protocols from GitHub...
✅ Found 1 protocols.
📝 Please enter your details below (first initial and last name, e.g., AOnoufriou).


Text(value='', description='Your Name:', placeholder='e.g., AOnoufriou', style=DescriptionStyle(description_wi…

IntText(value=50, description='Sample size:', style=DescriptionStyle(description_width='initial'))

Dropdown(description='Protocol:', layout=Layout(width='95%'), options=('test_protocol_check/protocols/dna_extr…

Output()